# Modular CerberusTS Experimentation

In [1]:
import os
os.chdir("..")

In [2]:
from cerberus_ts import Cerberus, train_cerberus
from cerberus_ts import TimeseriesDataPreparer, ResponseGenerator
from cerberus_ts import Foresight, train_foresight

import pandas as pd

c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from cerberus_ts import CerberusConfig
CerberusConfig.set_masked_norm_zero = True

ImportError: cannot import name 'CerberusConfig' from 'cerberus_ts' (c:\Users\johnm\Documents\GitHub\cerberusPy\cerberus_ts\__init__.py)

In [ ]:
df = pd.read_csv(r"data/jena_climate_2009_2016.csv",
                parse_dates=['Date Time'], 
                index_col=['Date Time'])
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
df = df.iloc[:5000,:]

In [ ]:
thresholds = {
    'call': 1,
    'response': 1,
    'context_0': 1,
    'context_1': 1,
    'context_2': 1
}

sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

window_timesteps = {
    'call': '10T',
    'response': '10T',
    'context_0': '1H',
    'context_1': '2H',
    'context_2': '6H'
}

window_sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

feature_indexes = {
    'call': range(0,14),
    'response': [0, 1, 4],
    'context_0': range(0,14),
    'context_1': range(0,14),
    'context_2': range(0,14)   
}

In [ ]:
# Initialize the preparer
preparer = TimeseriesDataPreparer(df, sizes, thresholds, feature_indexes, window_timesteps, train_len = 20_000, feature_range = (0, 1), batch_size = 100)

# Prepare the data
preparer.prepare_data()

In [ ]:
foresight = Foresight(sizes,feature_indexes)

In [ ]:
#foresight = train_foresight(foresight, preparer.dataloaders, num_epochs = 3)

In [ ]:
model = Cerberus(sizes, feature_indexes)

In [ ]:
model = train_cerberus(model, preparer.dataloaders, num_epochs = 60)

In [ ]:
# Intialize Response Generator
generator = ResponseGenerator(model, preparer.sliced_data, feature_indexes, preparer.max_change_dfs)

# Generate a response for a specific index
sel_index = 123
generator.generate_response(sel_index)

print(generator.denormalized_response)
generator.plot_normalized_responses()

In [ ]:
iterators = [iter(dataloader) for dataloader in preparer.dataloaders]
batches = [next(iterator) for iterator in iterators]
responses_batch = next(batch[2] for batch in batches)
print(responses_batch[0,0,:,:])
print(responses_batch[0,1,:,:])

In [ ]:
generator.plot_unscaled_responses(preparer.min_max_df, feature_indexes)